# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [4]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [5]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [6]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [7]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [8]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [9]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [10]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [11]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [12]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [13]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/mistralai/Magistral-Small-2506',
 '/nanonets/Nanonets-OCR-s',
 '/echo840/MonkeyOCR',
 '/tencent/Hunyuan3D-2.1',
 '/openbmb/MiniCPM4-8B',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/ResembleAI/Chatterbox',
 '/spaces/aisheets/sheets',
 '/spaces/multimodalart/wan2-1-fast',
 '/spaces/Agents-MCP-Hackathon/AI-Marketing-Content-Creator',
 '/spaces',
 '/datasets/nvidia/Nemotron-Personas',
 '/datasets/institutional/institutional-books-1.0',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/open-thoughts/OpenThoughts3-1.2M',
 '/datasets/miriad/miriad-5.8M',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs

In [14]:
get_links("https://huggingface.co")

{'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'},
  {'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'status page', 'url': 'https://status.huggingface.co/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [15]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [16]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
mistralai/Magistral-Small-2506
Updated


In [17]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [18]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [19]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmistralai/Magistral-Small-2506\nUpdated\nabout 10 hours ago\n•\n13.5k\n•\n429\nnanonets/Nanonets-OCR-s\nUpdated\n1 day ago\n•\n7.96k\n•\n276\necho840/MonkeyOCR\nUpdated\n4 days ago\n•\n4\n•\n262\ntencent/Hunyuan3D-2.1\nUpdated\nabout 23 hours ago\n•\n2.75k\n•\n207\nopenbmb/MiniCPM4-8B\nUpdated\n3 days ago\n•\n7.04k\n•\n244\nBrowse 1M+ models\nSpaces\nRunning\n8.09k\n8.09k\nDeepSite\n🐳\nGen

In [20]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [21]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blogs page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


# Hugging Face: The AI Community Building the Future

## Who We Are
Hugging Face is an innovative company dedicated to democratizing artificial intelligence (AI) and machine learning (ML) for everyone. Our mission is to create a productive and collaborative environment where developers, researchers, and organizations can build state-of-the-art ML models, access diverse datasets, and deploy applications that push the boundaries of technology.

## What We Offer
- **Models**: Explore a vast library of over **1 million models**, including the latest machine learning advancements.
- **Datasets**: Access and share **over 250,000 datasets** designed for various ML tasks.
- **Spaces**: Utilize our platform for collaborative development, allowing users to create and run applications, enhancing innovation.

### Our Services
- **Compute**: Optimize your ML projects with our paid compute solutions starting at **$0.60/hour for GPU**.
- **Enterprise Solutions**: Tailored for organizations seeking enterprise-grade security, support, and access controls starting at **$20/user/month**.

## Our Community
Join a thriving community of **more than 50,000 organizations**, including notable names like Amazon, Google, Microsoft, and Grammarly. Hugging Face fosters collaboration across various sectors, emphasizing the importance of shared knowledge and development efforts.

## Company Culture
At Hugging Face, we embrace a culture of openness and inclusivity, where every team member can contribute to the advancement of machine learning technology. Our community-driven approach nurtures innovation, collaboration, and shared learning, making it a great place for AI enthusiasts and professionals alike.

## Career Opportunities
We are always on the lookout for passionate individuals to join our diverse team. Whether you're an entry-level candidate or a seasoned professional, your journey in AI and machine learning can start here. Check our **[Jobs page](https://huggingface.co/jobs)** for current openings.

## Engage with Us
Stay connected and be a part of our AI revolution:
- **Follow us** on [Twitter](https://twitter.com/huggingface), [LinkedIn](https://www.linkedin.com/company/huggingface/), and [Discord](https://discord.gg/huggingface)
- Explore our latest innovations and discussions on the [Hugging Face Blog](https://huggingface.co/blog)

Join us today and help build the future of AI!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [22]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [23]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face Brochure

## Welcome to Hugging Face – The AI Community Building the Future

At Hugging Face, we are on a mission to democratize artificial intelligence by creating a collaborative platform for the machine learning community. Our ecosystem is designed for developers, researchers, and enterprises to explore, create, and share machine learning models, datasets, and applications.

### Why Choose Hugging Face?

- **Collaboration**: Join a community of over 50,000 organizations leveraging our platform to enhance their machine learning capabilities. Collaborate on over 1 million models and 250,000 datasets.
- **State-of-the-Art Tools**: We offer a range of advanced open-source tools and libraries, including Transformers and Diffusers, providing the best solutions for researchers and developers alike.
- **Flexible Solutions**: Whether you are an independent developer or part of a large enterprise, we provide scalable compute solutions starting at just $0.60/hour for GPU access.

### Platforms and Products

- **Models**: Access and contribute to a vast library of cutting-edge machine learning models.
- **Datasets**: Browse and utilize a diverse selection of datasets for various machine learning tasks.
- **Spaces**: Create and deploy applications using our user-friendly Spaces, perfect for testing and sharing ideas with the community.

### Company Culture

Our team is committed to openness and collaboration. We believe in fostering an environment where creativity thrives, and individuals can take risks to innovate. We advocate for inclusivity and support diverse voices within the AI community.

### Career Opportunities

Join us and become part of a dynamic team! At Hugging Face, we are continuously looking for passionate individuals to help shape the future of AI. Explore our [current openings](#) to see how you can contribute to our mission.

### Trusted by Leading Organizations

Our platform has been embraced by reputable companies such as Amazon, Google, Microsoft, and Grammarly, all using Hugging Face to enhance their artificial intelligence workflows.

---

Explore your potential with Hugging Face, where you can create, collaborate, and innovate within the ever-evolving world of AI. 

For more information, visit [HuggingFace.com](https://huggingface.co) and join our vibrant community!

In [24]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## 🌟 Welcome to Hugging Face

### The AI Community Building the Future

At Hugging Face, we empower the machine learning community to collaborate and innovate. Our platform is the go-to destination for models, datasets, and applications, fostering a culture of open-source collaboration.

---

## 🌐 Explore Our Offerings

- **Models:** Access over **1 million models** catering to various tasks in AI.
- **Datasets:** Browse through **250,000+ datasets** for every machine learning need.
- **Spaces:** Interact with **400k+ applications** that leverage our powerful tools.
- **Enterprise Solutions:** Tailored solutions starting at **$20/user/month** for organizations looking to build AI with enterprise-grade security.

---

## 🤝 Community Collaboration

With more than **50,000 organizations** using Hugging Face, including major players like Google, Microsoft, and Amazon, our community is redefining how AI is built and shared. We encourage sharing and collaboration, allowing individuals and organizations to build their ML profiles and portfolios.

---

## 💼 Join Our Team

Hugging Face is not just about technology; it's about people! We are always looking for passionate individuals to join our growing team. Check out our current openings and discover a workplace that values collaboration, innovation, and diversity.

---

## 🌱 Company Culture

At Hugging Face, we believe in creating a supportive environment that fosters creativity and collaboration. Our team values transparency and open communication, encouraging everyone to contribute ideas and insights. We are committed to building a diverse and inclusive workplace that reflects the community we serve.

---

## 📈 Invest in the Future of AI

We are on a mission to democratize AI and ML technologies through an open-source approach. Our enterprise solutions provide the tools required to develop cutting-edge technologies securely and efficiently. By joining us, you’re partnering in a venture that is shaping the future of AI and machine learning.

---

### 🗣️ Connect with Us

- **Website:** [Hugging Face](https://huggingface.co)
- **Social Media:** Follow us on Twitter, LinkedIn, and Discord for updates and discussions.

--- 

Join us at Hugging Face, where the community and innovation meet to build the AI of tomorrow!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>